In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats as st
import math
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号 #有中文出现的情况

In [22]:
class LogitRegression:
    
    class Model:
        """
        desc:训练出的模型
        """
        def __init__(self,theta):
            """
            desc:根据传入的 θ 构建模型 
            """
            self.theta = theta
        def predict(self,x_test):
            """
            desc:根据传入的特征,利用模型预测数据
            """
            t = np.dot(x_test, self.theta)
            # 得出类别为 1 的概率
            result = LogitRegression.sigmoid(t)
            # 将所有数据取整  p > 0.5 return: 1 , p <= 0.5 return: 0
            return np.round(result)
    
    def __init__(self,theta,lamb,Sigma,mu,sig,max_iter):
        ''' 
        :param theta: 待估计的参数向量（logit回归参数）
        :param lamb: L1正则的超参数
        :param sigma: 用于生成eps的多元正态分布的协方差矩阵
        :param mu: 先验正态分布的均值
        :param sig: 先验正态分布的协方差矩阵的对角元素
        :param max_iter: MCMC的最大迭代次数
        '''
        self.theta = theta
        self.lamb = lamb
        self.Sigma = Sigma
        self.mu = mu
        self.sig = sig
        self.max_iter = max_iter

    def fit(self,x_data,y_data,method):
        """
        :param x_data: 特征值
        :param y_data: 标签值
        :param method: 方法l1或l2
        """
        iter_cnt = 0  #初始化迭代次数为0
        data_len = x_data.shape[0]  #样本量n
        # 开始迭代
        if method=='l1':
            self.l1_logit_mcmc(x_data,y_data)
        elif method=='l2':
            self.l2_logit_mcmc(x_data,y_data)
        else:
            print('You use the wrong method.')

        return self.Model(self.theta)
    
    def sigmoid(x):
        t = 1 + np.exp(-x)
        result = np.divide(1,t)
        return result

    def l1_logit_func(self,X,y,thetas):
        ''' 
        :param X: 待输入的数据特征
        :param y: 待输入的数据标签
        :param theta: 待估计的参数向量（logit回归参数）
        :param lamb: L1正则的超参数
        :param mu: 先验正态分布的均值
        :param sig: 先验正态分布的协方差矩阵的对角元素
        '''
        data_len = X.shape[0]  #样本量n
        feature_len = thetas.shape[0]  #特征数p
        j_ = 1
        for i in range(data_len):
            t1 = np.dot(X[i], thetas) 
            h_theta = LogitRegression.sigmoid(t1)
            h_ = pow(h_theta,y[i])*pow(1-h_theta,1-y[i])
            j_ = j_*h_
        
        k_ = 1
        for j in range(feature_len):
            p_ = np.exp(-np.abs(thetas[j]-self.mu[j])/self.sig[j])/(2*self.sig[j])
            k_ = k_*p_

        dis = j_*k_
        return dis

    def l2_logit_func(self,X,y,thetas):
        ''' 
        :param X: 待输入的数据特征
        :param y: 待输入的数据标签
        :param theta: 待估计的参数向量（logit回归参数）
        :param lamb: L1正则的超参数
        :param mu: 先验正态分布的均值
        :param sig: 先验正态分布的协方差矩阵的对角元素
        '''
        data_len = X.shape[0]  #样本量n
        feature_len = thetas.shape[0]  #特征数p
        j_ = 1
        for i in range(data_len):
            t1 = np.dot(X[i], thetas) 
            h_theta = LogitRegression.sigmoid(t1)
            h_ = pow(h_theta,y[i])*pow(1-h_theta,1-y[i])
            j_ = j_*h_
        
        k_ = 1
        for j in range(feature_len):
            p_ = np.exp(-pow(thetas[j]-self.mu[j],2)/(2*pow(self.sig[j],2)))/(math.sqrt(2*math.pi)*self.sig[j])
            k_ = k_*p_

        dis = j_*k_
        return dis

    def l1_logit_mcmc(self,X,y):
        
        ''' 
        :param X: 待输入的数据特征
        :param y: 待输入的数据标签
        :param theta: 待估计的参数向量（logit回归参数）
        :param lamb: L1正则的超参数
        :param sigma: 用于生成eps的多元正态分布的协方差矩阵
        :param mu: 先验正态分布的均值
        :param sig: 先验正态分布的协方差矩阵的对角元素
        :param max_iter: MCMC的最大迭代次数
        '''
        theta_list = []
        theta_list.append(self.theta)
        accept = 0
        feature_len = self.theta.shape[0]  #特征数p
        for i in np.arange(1,self.max_iter):
            beta = np.random.multivariate_normal(theta_list[i-1],self.Sigma,size=1).squeeze()
            u = np.random.rand(1)
            judge = self.l1_logit_func(X,y,beta)/self.l1_logit_func(X,y,theta_list[i-1])
            prob = min(1,judge)
            if u <= prob:
                theta_list.append(beta)
                accept += 1
            else:
                theta_list.append(theta_list[i-1])

        accept_rate = accept/self.max_iter
        self.theta = theta_list[-1]
        return theta_list,accept_rate

    def l2_logit_mcmc(self,X,y):
        
        ''' 
        :param X: 待输入的数据特征
        :param y: 待输入的数据标签
        :param theta: 待估计的参数向量（logit回归参数）
        :param lamb: L1正则的超参数
        :param sigma: 用于生成eps的多元正态分布的协方差矩阵
        :param mu: 先验正态分布的均值
        :param sig: 先验正态分布的协方差矩阵的对角元素
        :param max_iter: MCMC的最大迭代次数
        '''
        theta_list = []
        theta_list.append(self.theta)
        accept = 0
        feature_len = self.theta.shape[0]  #特征数p
        for i in np.arange(1,self.max_iter):
            beta = np.random.multivariate_normal(theta_list[i-1],self.Sigma,size=1).squeeze()
            u = np.random.rand(1)
            judge = self.l2_logit_func(X,y,beta)/self.l2_logit_func(X,y,theta_list[i-1])
            prob = min(1,judge)
            if u <= prob:
                theta_list.append(beta)
                accept += 1
            else:
                theta_list.append(theta_list[i-1])

        accept_rate = accept/self.max_iter
        self.theta = theta_list[-1]
        return theta_list,accept_rate

#### 用鸢尾花数据集进行验证

In [30]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
iris_data = np.column_stack(load_iris(return_X_y=True))
# 这里我们实现的只是二分类算法,左右只需要类别0和类别1两种即可
iris_data = iris_data[iris_data[:,-1]<2]
# 将数据打乱
np.random.shuffle(iris_data)
# 将数据分成两份
train_x,test_x,train_y,test_y = train_test_split(iris_data[:,:-1],iris_data[:,-1],test_size=0.3,random_state=2)

In [25]:
ini_theta = np.array([0]*train_x.shape[1],dtype=float)
sigma = np.eye(ini_theta.shape[0])
mu = np.array([0]*train_x.shape[1],dtype=float)
sig = np.array([1]*train_x.shape[1],dtype=float)
lr = LogitRegression(theta=ini_theta,lamb=0.01,Sigma=sigma,mu=mu,sig=sig,max_iter=30000)
m = lr.fit(train_x,train_y,'l1')
pred_y_l1 = m.predict(test_x)

acc = accuracy_score(test_y,pred_y_l1)
rec = recall_score(test_y,pred_y_l1)
pre = precision_score(test_y,pred_y_l1)
f1 = f1_score(test_y,pred_y_l1)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))
print('模型系数结果beta为',m.theta)

预测准确率为:1.0000
预测查准率为:1.0000
预测召回率为:1.0000
预测f1-score为:1.0000
模型系数结果beta为 [-0.97252456 -1.06642838  3.81230857 -0.64347327]


In [26]:
ini_theta = np.array([0]*train_x.shape[1],dtype=float)
sigma = np.eye(ini_theta.shape[0])
mu = np.array([0]*train_x.shape[1],dtype=float)
sig = np.array([1]*train_x.shape[1],dtype=float)
lr = LogitRegression(theta=ini_theta,lamb=0.01,Sigma=sigma,mu=mu,sig=sig,max_iter=30000)
m = lr.fit(train_x,train_y,'l2')
pred_y_l2 = m.predict(test_x)

acc = accuracy_score(test_y,pred_y_l2)
rec = recall_score(test_y,pred_y_l2)
pre = precision_score(test_y,pred_y_l2)
f1 = f1_score(test_y,pred_y_l2)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))
print('模型系数结果beta为',m.theta)

预测准确率为:1.0000
预测查准率为:1.0000
预测召回率为:1.0000
预测f1-score为:1.0000
模型系数结果beta为 [-0.6452443  -1.26659608  2.69012003  0.52950857]


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

lr_l1 = LogisticRegression(penalty="l2", solver="liblinear", max_iter=1000)
lr_l1.fit(train_x,train_y)
pred_y_lr = lr_l1.predict(test_x)

acc = accuracy_score(test_y,pred_y_lr)
rec = recall_score(test_y,pred_y_lr)
pre = precision_score(test_y,pred_y_lr)
f1 = f1_score(test_y,pred_y_lr)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))

print(lr_l1.coef_)

预测准确率为:1.0000
预测查准率为:1.0000
预测召回率为:1.0000
预测f1-score为:1.0000
[[-0.39576795 -1.33967636  2.08088193  0.9478365 ]]


#### 用作业五数据集进行验证

In [33]:
data = pd.read_csv('data.csv')

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 训练数据
train_x = train_data.iloc[:,1:].values 
train_y = np.array(train_data.iloc[:,0])
# 测试数据
test_x = test_data.iloc[:,1:].values
test_y = np.array(test_data.iloc[:,0])

In [21]:
ini_theta = np.array([0]*train_x.shape[1],dtype=float)
sigma = np.eye(ini_theta.shape[0])
mu = np.array([0]*train_x.shape[1],dtype=float)
sig = np.array([1]*train_x.shape[1],dtype=float)
lr = LogitRegression(theta=ini_theta,lamb=0.01,Sigma=sigma,mu=mu,sig=sig,max_iter=10000)
m = lr.fit(train_x,train_y,'l1')
pred_y_l1 = m.predict(test_x)

acc = accuracy_score(test_y,pred_y_l1)
rec = recall_score(test_y,pred_y_l1)
pre = precision_score(test_y,pred_y_l1)
f1 = f1_score(test_y,pred_y_l1)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))
print('模型系数结果beta为',m.theta)

预测准确率为:0.7000
预测查准率为:0.6667
预测召回率为:0.7368
预测f1-score为:0.7000
模型系数结果beta为 [ 1.00319243  2.2100314   1.02455745 -2.01142384 -2.04800168]


In [29]:
ini_theta = np.array([0]*train_x.shape[1],dtype=float)
sigma = np.eye(ini_theta.shape[0])
mu = np.array([0]*train_x.shape[1],dtype=float)
sig = np.array([1]*train_x.shape[1],dtype=float)
lr = LogitRegression(theta=ini_theta,lamb=0.01,Sigma=sigma,mu=mu,sig=sig,max_iter=30000)
m = lr.fit(train_x,train_y,'l2')
pred_y_l2 = m.predict(test_x)

acc = accuracy_score(test_y,pred_y_l2)
rec = recall_score(test_y,pred_y_l2)
pre = precision_score(test_y,pred_y_l2)
f1 = f1_score(test_y,pred_y_l2)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))
print('模型系数结果beta为',m.theta)

预测准确率为:0.7750
预测查准率为:0.7273
预测召回率为:0.8421
预测f1-score为:0.7805
模型系数结果beta为 [ 1.14342195  1.31200235  0.02890504 -0.55291776 -1.95797937]


In [20]:
lr_l1 = LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000)
lr_l1.fit(train_x,train_y)
pred_y_lr = lr_l1.predict(test_x)

acc = accuracy_score(test_y,pred_y_lr)
rec = recall_score(test_y,pred_y_lr)
pre = precision_score(test_y,pred_y_lr)
f1 = f1_score(test_y,pred_y_lr)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))

print(lr_l1.coef_)

预测准确率为:0.7250
预测查准率为:0.6667
预测召回率为:0.8421
预测f1-score为:0.7442
[[ 1.20252945  1.42441513  0.40335074 -1.10683269 -1.9855369 ]]


In [34]:
lr_l1 = LogisticRegression(penalty="l2", solver="liblinear", max_iter=1000)
lr_l1.fit(train_x,train_y)
pred_y_lr = lr_l1.predict(test_x)

acc = accuracy_score(test_y,pred_y_lr)
rec = recall_score(test_y,pred_y_lr)
pre = precision_score(test_y,pred_y_lr)
f1 = f1_score(test_y,pred_y_lr)

print('预测准确率为:{:.4f}'.format(acc))
print('预测查准率为:{:.4f}'.format(pre))
print('预测召回率为:{:.4f}'.format(rec))
print('预测f1-score为:{:.4f}'.format(f1))

print(lr_l1.coef_)

预测准确率为:0.7000
预测查准率为:0.6522
预测召回率为:0.7895
预测f1-score为:0.7143
[[ 1.18490769  1.2798258   0.47955201 -1.17427586 -1.82474141]]
